<a href="https://colab.research.google.com/github/YuxinZhang1998/S3-SIF/blob/main/S3_features_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import datetime

# Authenticate only once per session
ee.Authenticate()
ee.Initialize(project='ee-yuxinzhanguv1')

def process_s3_olci(
    start_date: str,
    end_date: str,
    time_window: int,
    pixel_size: int,
    export_asset_folder: str,
    aoi: ee.Geometry,
    bands: list = None,
):
    """
    Process Sentinel-3 OLCI images into composites, compute vegetation indices,
    and export them to Google Earth Engine assets.

    Args:
        start_date (str): Start date in 'YYYY-MM-DD'.
        end_date (str): End date in 'YYYY-MM-DD'.
        time_window (int): Number of days per composite.
        pixel_size (int): Spatial resolution in meters.
        export_asset_folder (str): Asset folder to export results.
        aoi (ee.Geometry): Region of interest (default: global extent).
        bands (list): Radiance bands to include (default OLCI set).
    """

    if bands is None:
        bands = [
            'Oa01_radiance','Oa02_radiance','Oa03_radiance','Oa04_radiance',
            'Oa05_radiance','Oa06_radiance','Oa07_radiance','Oa08_radiance',
            'Oa09_radiance','Oa10_radiance','Oa11_radiance','Oa12_radiance',
            'Oa13_radiance','Oa15_radiance','Oa17_radiance','Oa18_radiance'
        ]

    # Convert dates
    startDate = datetime.datetime.strptime(start_date, "%Y-%m-%d").date()
    endDate = datetime.datetime.strptime(end_date, "%Y-%m-%d").date()
    startDateGEE = ee.Date(start_date)
    totalWindows = (endDate - startDate).days // time_window

    # Load Sentinel-3 OLCI collection
    S3Olci = ee.ImageCollection('COPERNICUS/S3/OLCI').filterDate(start_date, end_date)

    # --- Functions ---
    def maskS3badPixels(image):
        qa = image.select('quality_flags')
        # Quality mask bits
        land = 1 << 31
        coastLine = 1 << 30
        inLandWater = 1 << 29
        bright = 1 << 27
        invalid = 1 << 25
        straylight = 1 << 26
        cosmetic = 1 << 24
        dubious = 1 << 21
        # Saturated band flags
        Oa12Sat = 1 << 9; Oa13Sat = 1 << 8; Oa17Sat = 1 << 4; Oa18Sat = 1 << 3
        Oa15Sat = 1 << 6; Oa10Sat = 1 << 11; Oa03Sat = 1 << 18; Oa06Sat = 1 << 15

        mask = qa.bitwiseAnd(land).neq(0) \
            .And(qa.bitwiseAnd(coastLine).eq(0)) \
            .And(qa.bitwiseAnd(inLandWater).eq(0)) \
            .And(qa.bitwiseAnd(bright).eq(0)) \
            .And(qa.bitwiseAnd(invalid).eq(0)) \
            .And(qa.bitwiseAnd(straylight).eq(0)) \
            .And(qa.bitwiseAnd(cosmetic).eq(0)) \
            .And(qa.bitwiseAnd(dubious).eq(0)) \
            .And(qa.bitwiseAnd(Oa12Sat).eq(0)) \
            .And(qa.bitwiseAnd(Oa13Sat).eq(0)) \
            .And(qa.bitwiseAnd(Oa17Sat).eq(0)) \
            .And(qa.bitwiseAnd(Oa18Sat).eq(0)) \
            .And(qa.bitwiseAnd(Oa15Sat).eq(0)) \
            .And(qa.bitwiseAnd(Oa10Sat).eq(0)) \
            .And(qa.bitwiseAnd(Oa03Sat).eq(0)) \
            .And(qa.bitwiseAnd(Oa06Sat).eq(0))

        return image.updateMask(mask).select(bands).toFloat()

    def getInputDates(i):
        fecha_inicio = startDateGEE.advance(i * time_window, 'day')
        fecha_fin = fecha_inicio.advance(time_window, 'day')
        fecha_str = datetime.datetime.utcfromtimestamp(fecha_inicio.getInfo()['value'] / 1000.0).strftime('%Y%m%d')
        return {'start': fecha_inicio, 'end': fecha_fin, 'label': fecha_str}

    def computeIndices(img):
        all_needed_bands = bands
        img = img.select(all_needed_bands).toFloat()
        # k
        k = img.select('Oa10_radiance').divide(img.select('Oa15_radiance')).rename('k')
        # L
        L = img.select('Oa12_radiance').subtract(
                img.select('Oa13_radiance').add(img.select('Oa15_radiance')).multiply(0.5)
            ).rename('L')
        # Radmean
        rad_bands = [f'Oa{i:02d}_radiance' for i in range(1, 12)]
        radmean = img.select(rad_bands).reduce(ee.Reducer.mean()).rename('Radmean')
        # NDVI
        ndvi = img.normalizedDifference(['Oa17_radiance', 'Oa08_radiance']).rename('NDVI')
        # NIRv
        nirv = ndvi.multiply(img.select('Oa17_radiance')).rename('NIRv')
        original = img.select(['Oa12_radiance', 'Oa13_radiance', 'Oa17_radiance', 'Oa18_radiance'])
        return img.addBands([k, L, radmean, ndvi, nirv, original])

    # --- Processing loop ---
    composite_collection = ee.ImageCollection([])
    for i in range(totalWindows):
        dates = getInputDates(i)
        print(f"Processing window: {dates['label']}")
        composite = (S3Olci
            .filterDate(dates['start'], dates['end'])
            .filterBounds(aoi)
            .map(maskS3badPixels)
            .select(bands)
            .mean()
            .reproject(crs='EPSG:4326', scale=pixel_size)
            .clip(aoi)
            .set('system:time_start', dates['start'].millis())
        )
        composite_collection = composite_collection.merge(ee.ImageCollection([composite]))

    # Compute indices
    collection_S3 = composite_collection.map(computeIndices)
    collection_S3_feature = collection_S3.select([
        'Oa12_radiance', 'Oa13_radiance', 'Oa17_radiance', 'Oa18_radiance',
        'k', 'L', 'Radmean', 'NDVI', 'NIRv'
    ])

    # Export each image
    imageList = collection_S3_feature.toList(collection_S3_feature.size())
    n_images = collection_S3_feature.size().getInfo()

    for i in range(n_images):
        try:
            image = ee.Image(imageList.get(i)).toFloat()
            date_str = ee.Date(image.get('system:time_start')).format('YYYYMMdd').getInfo()
            export_name = f"S3_export_{date_str}"
            asset_path = f"{export_asset_folder}/{export_name}"

            print(f"Exporting: {export_name}")
            task = ee.batch.Export.image.toAsset(
                image=image,
                description=export_name,
                assetId=asset_path,
                region=aoi,
                scale=pixel_size
            )
            task.start()
        except Exception as e:
            print(f"Error exporting image {i}: {e}")
            continue

    print("All export tasks have been created.")



*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [ ]:
# Example usage
aoi_global = ee.Geometry.Rectangle([-10, 36, 4, 44], geodesic=False);

process_s3_olci(
    start_date="2019-01-01",
    end_date="2019-12-31",
    time_window=8,
    pixel_size=300,
    export_asset_folder="projects/ee-yuxinzhanguv1/assets/S3-8DAYS-peninsula2019-300-FeatureCollection",
    aoi=aoi_global
)

/tmp/ipython-input-1431261278.py:86: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  fecha_str = datetime.datetime.utcfromtimestamp(fecha_inicio.getInfo()['value'] / 1000.0).strftime('%Y%m%d')


Processing window: 20190101
Processing window: 20190109
Processing window: 20190117
Processing window: 20190125
Processing window: 20190202
Processing window: 20190210
Processing window: 20190218
Processing window: 20190226
Processing window: 20190306
Processing window: 20190314
Processing window: 20190322
Processing window: 20190330
Processing window: 20190407
Processing window: 20190415
Processing window: 20190423
Processing window: 20190501
Processing window: 20190509
Processing window: 20190517
Processing window: 20190525
Processing window: 20190602
Processing window: 20190610
Processing window: 20190618
Processing window: 20190626
Processing window: 20190704
Processing window: 20190712
Processing window: 20190720
Processing window: 20190728
Processing window: 20190805
Processing window: 20190813
Processing window: 20190821
Processing window: 20190829
Processing window: 20190906
Processing window: 20190914
Processing window: 20190922
Processing window: 20190930
Processing window: 2